In [18]:
import yfinance as yf
from datetime import datetime
import quandl
import stockquotes
import numpy as np
import scipy.stats as st 
import pandas as pd 
import matplotlib.pyplot as plt

%matplotlib inline

In [24]:
def Spot(ticker):
    '''
    Takes a ticker symbol and returns the last price of the asset
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    Returns:
    S -- Spot price of the asset
    '''
    
    security = stockquotes.Stock(ticker)
    S = security.current_price

    return S    

In [25]:
S = Spot("^GSPC")
print(f'S&P500 last price was ${S}, thus we will look for strike price in the range of ${round(S-300)} - ${round(S+300)}')

S&P500 last price was $3663.46, thus we will look for strike price in the range of $3363 - $3963


In [20]:
def call_data(ticker,expiration):
    '''
    Extracts the strike price and call premium of an option given the ticker symbol of the underlying asset and 
    the expiration date. Returns a list of strike price and call premium.
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset
                  
    Returns:
    data -- list containig pairs of strike price and call premiums available for the underlying in the expiration selected
    '''
    
    security = yf.Ticker(ticker)
    
    if expiration not in security.options:                       #Assertion to verify that expiration exists
        print('Option expiration not available, please verify')
        return
        
    options = security.option_chain(expiration)
    calls = options.calls
    strike_price = calls['strike']
    call_premium = calls['lastPrice']
    
    data = list(zip(strike_price, call_premium))
    
    return data

In [14]:
call_data('^GSPC','2021-12-16')

[(900.0, 2755.29),
 (1000.0, 2656.54),
 (1225.0, 1956.5),
 (1300.0, 2039.26),
 (1375.0, 2143.78),
 (1400.0, 2119.6),
 (1500.0, 2160.15),
 (1525.0, 1425.66),
 (1550.0, 1201.5),
 (1600.0, 1600.8),
 (1625.0, 1375.0),
 (1650.0, 1054.5),
 (1800.0, 1243.1),
 (1825.0, 1466.0),
 (1900.0, 1361.4),
 (2000.0, 1638.0),
 (2175.0, 779.5),
 (2200.0, 1166.64),
 (2250.0, 1311.3),
 (2300.0, 1321.8),
 (2350.0, 834.0),
 (2400.0, 1007.0),
 (2425.0, 760.89),
 (2475.0, 1074.55),
 (2500.0, 1205.23),
 (2525.0, 575.4),
 (2550.0, 877.68),
 (2575.0, 895.15),
 (2600.0, 1043.62),
 (2625.0, 920.0),
 (2650.0, 909.5),
 (2675.0, 564.9),
 (2700.0, 960.0),
 (2725.0, 756.5),
 (2750.0, 966.06),
 (2775.0, 371.06),
 (2800.0, 945.42),
 (2825.0, 814.86),
 (2850.0, 851.1),
 (2900.0, 813.82),
 (2925.0, 629.0),
 (2950.0, 797.0),
 (3000.0, 756.39),
 (3100.0, 670.9),
 (3200.0, 622.93),
 (3300.0, 550.0),
 (3400.0, 480.0),
 (3500.0, 386.35),
 (3600.0, 322.96),
 (3900.0, 158.66),
 (4000.0, 115.0)]

In [5]:
def Risk_free(ttm):
    '''
    Extracts the most recent US treasury rate given the time to maturity.
    
    Arguments:
    ttm -- Time to maturity in string format. The available options are:
    ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    Returns:
    rf -- Most recent risk free rate for the time to maturity selected
    '''
    yield_curve = quandl.get('USTREASURY/YIELD', authtoken='JMxryiBcRV26o9r5q7uv')
    rf = yield_curve[ttm][-1]
    
    return rf    

In [6]:
Risk_free('1 YR')

0.1

In [7]:
def Time_to_maturity(expiration):
    '''
    Calculates the time to maturity in years from today to the option expiration
    
    Arguments:
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset
                  
    Returns:
    ttm -- Time to maturity in year in float format
    '''
    
    today = datetime.now()
    expiration_d = datetime.strptime(expiration,'%Y-%m-%d')
    datetime_delta = (expiration_d - today).days
    ttm = datetime_delta/365
    
    return ttm
    

In [8]:
Time_to_maturity('2021-12-16')

1.010958904109589

In [82]:
def Norm_d1(ticker, expiration, ttm, sigma = 0.1):
    
    S = Spot(ticker)
    
    data = call_data(ticker,expiration)
    
    K, call_premium = list(zip(*data))
    
    S = np.array(S)
    K = np.array(K)
    
    r = Risk_free(ttm)
    
    T = Time_to_maturity(expiration)
    
    
    d1 = (np.log(np.divide(S,K))+(r+((np.power(sigma,2)/2))*T))/(sigma*np.sqrt(T))
    N_d1 = st.norm.cdf(d1)
    
    cache = (N_d1, T, r, S, K, call_premium)
    
    return d1, cache

In [53]:
Norm_d1('^GSPC','2021-12-16','1 YR')

(array([15.00623179, 13.95835279, 11.93997382, 11.34896917, 10.79112287,
        10.61191709,  9.92573802,  9.76134333,  9.59962186,  9.28386036,
         9.12966112,  8.97781616,  8.1124313 ,  7.97524772,  7.57469755,
         7.0645523 ,  6.23029627,  6.11663044,  5.89312324,  5.67452869,
         5.46063546,  5.25124559,  5.14818092,  4.94520139,  4.84524425,
         4.74628172,  4.64829421,  4.55126269,  4.45516869,  4.35999426,
         4.26572198,  4.17233489,  4.07981653,  3.98815089,  3.89732239,
         3.80731587,  3.71811661,  3.62971023,  3.54208278,  3.36911056,
         3.28373963,  3.19909528,  3.03193753,  2.70582137,  2.39005987,
         2.08401567,  1.7871085 ,  1.49880855,  1.21863082,  0.42255392,
         0.17075182]),
 (array([1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.     

In [84]:
def Norm_d2(ticker, expiration, ttm, sigma = 0.1):
    
    d1, cache = Norm_d1(ticker, expiration, ttm, sigma)
    
    N_d1, T, r, S, K, call_premium = cache
    
    d2 = np.subtract(d1,sigma*np.sqrt(T))
    N_d2 = st.norm.cdf(d2)
    
    cache = (N_d1, S, K, r, T, call_premium)
    
    return N_d2, cache

In [57]:
Norm_d2('^GSPC','2021-12-16','1 YR')

(array([1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 0.99999999,
        0.99999996, 0.99999987, 0.99999978, 0.99999937, 0.99999896,
        0.99999831, 0.99999729, 0.99999572, 0.99999334, 0.99998975,
        0.99998444, 0.99997667, 0.99996544, 0.99994938, 0.9999267 ,
        0.99989504, 0.99985131, 0.99979156, 0.99971079, 0.99945953,
        0.9992717 , 0.99902765, 0.99831276, 0.99540997, 0.98897523,
        0.97634248, 0.9541562 , 0.91898281, 0.86823452, 0.62627648,
        0.5279849 ]),
 (array([1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 0.99999999,
         0.99999998, 0

In [87]:
def Call_price(ticker, expiration, ttm, sigma = 0.1):
    
    N_d2, cache = Norm_d2(ticker, expiration, ttm, sigma)
    
    N_d1, S, K, r, T, call_premium = cache
    
    C = np.subtract(np.multiply(S,N_d1),np.multiply(K,np.multiply(np.exp(-r*T),N_d2)))
    
    call_premium = np.array(call_premium)
    
    return C, call_premium

In [ ]:
from scipy.optimize import minimize

In [115]:
def difference(ticker, expiration, ttm, sigma = 0.1):
    
    B_S, call_premium = Call_price(ticker, expiration, ttm, sigma)
    
    difference = np.subtract(B_S, call_premium)
    
    return difference